**Part 1: Wrangling our datasets**
Taking a look into both dataset given and wrangling them so they are tidy.  
Load our R packages

In [ ]:
library(tidyverse)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(forcats)
library(tidymodels)

**Reading our datasets**

In [ ]:
players <- read_csv("data/players.csv")
players
sessions <- read_csv("data/sessions.csv")
sessions

Focusing on the first player dataset, I want to select the variables that aligns to my question and goal.

In [ ]:
players_select <- players|>
    select(experience, hashedEmail, played_hours, Age) |>
    filter(Age != "NA")|>
    filter(played_hours != 0.0)|>
    mutate(experience = as.factor(experience))
players_select

In [ ]:
avg_playtime<- players_select|>
group_by(experience)|>
summarize(avg_play = median(played_hours))
avg_playtime
experience_count<- players_select|>
group_by(experience)|>
summarize(count = n())
experience_count

In [ ]:
experience_bar<- avg_playtime |>
ggplot(aes(y = avg_play, x = fct_reorder(experience, avg_play), fill = experience))+
geom_bar(stat = "identity")+
labs(x = "Minecraft Experience", y = "Average Playtime (in hours)", fill = "Minecraft Experience") +
scale_fill_brewer(palette = "BrBG") +
ggtitle("Average Playtime for Different Players")
experience_bar

In [ ]:
age_experience_plot<- players_select|>
ggplot(aes(x= Age, y = played_hours, colour = experience))+
    geom_point(alpha = 0.5)+
    labs(x= "Age (years)", y = "Hours Played", colour = "Minecraft Experience")+
    ggtitle("Hours Played vs Age")
age_experience_plot

In [ ]:
set.seed(13)
players_split<- initial_split(players_select, prop = 0.75, strata = played_hours)
players_training<- training(players_split)
players_testing<- testing(players_split)

In [ ]:
players_knn <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
      set_engine("kknn") |>
      set_mode("regression") 

players_recipe <- recipe(played_hours ~ Age, data = players_training) |>
      step_scale(all_predictors()) |>
      step_center(all_predictors())

In [ ]:
players_vfold<- vfold_cv(players_training, v = 5, strata = played_hours)
players_workflow<- workflow()|>
    add_recipe(players_recipe)|>
    add_model(players_knn)
players_workflow

In [ ]:
gridvals <- tibble(neighbors = seq(from = 1, to = 51, by = 10))
players_results<- players_workflow|>
                    tune_grid(resamples = players_vfold, grid = gridvals)|>
                   collect_metrics()
players_results

In [ ]:
players_min <- players_results |>
   filter(.metric == "rmse") |>
   slice_min(mean, n = 1)
players_min

In [ ]:
k_min <- players_min |>
         pull(neighbors)

players_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) |>
         set_engine("kknn") |>
         set_mode("regression")

players_best_fit <- workflow() |>
         add_recipe(players_recipe) |>
         add_model(players_best_spec) |>
         fit(data = players_training)

players_summary <- players_best_fit |>
          predict(players_testing) |>
          bind_cols(players_testing) |>
          metrics(truth = played_hours, estimate = .pred)
players_summary

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)

# your code here
players_preds<- players_best_fit|>
    predict(players_training)|>
    bind_cols(players_training)
players_preds

    players_plot<- ggplot(player_preds, aes(x = max, y = time_hrs))+
        geom_point()+
        geom_line(data = player_preds, mapping = aes(x = Age, y = .pred), color = "blue")+
        labs(x = "Age", y = "Time Played (in hours)")+
        ggtitle("Time Played vs Age of Player")
players_plot